In [46]:
import h5py
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Masking, TimeDistributed
%matplotlib inline
import matplotlib.pyplot as plt
import random

In [102]:
data_path = '../data'
train_file = h5py.File(os.path.join(data_path, 'TRAIN', 'features.h5'), 'r')
test_file = h5py.File(os.path.join(data_path, 'TEST', 'features.h5'), 'r')

In [103]:
list(test_file.keys())

['DR1FAKS0SA1',
 'DR1FAKS0SA2',
 'DR1FAKS0SI1573',
 'DR1FAKS0SI2203',
 'DR1FAKS0SI943',
 'DR1FAKS0SX133',
 'DR1FAKS0SX223',
 'DR1FAKS0SX313',
 'DR1FAKS0SX403',
 'DR1FAKS0SX43',
 'DR1FDAC1SA1',
 'DR1FDAC1SA2',
 'DR1FDAC1SI1474',
 'DR1FDAC1SI2104',
 'DR1FDAC1SI844',
 'DR1FDAC1SX124',
 'DR1FDAC1SX214',
 'DR1FDAC1SX304',
 'DR1FDAC1SX34',
 'DR1FDAC1SX394',
 'DR1FELC0SA1',
 'DR1FELC0SA2',
 'DR1FELC0SI1386',
 'DR1FELC0SI2016',
 'DR1FELC0SI756',
 'DR1FELC0SX126',
 'DR1FELC0SX216',
 'DR1FELC0SX306',
 'DR1FELC0SX36',
 'DR1FELC0SX396',
 'DR1FJEM0SA1',
 'DR1FJEM0SA2',
 'DR1FJEM0SI1264',
 'DR1FJEM0SI1894',
 'DR1FJEM0SI634',
 'DR1FJEM0SX184',
 'DR1FJEM0SX274',
 'DR1FJEM0SX364',
 'DR1FJEM0SX4',
 'DR1FJEM0SX94',
 'DR1MDAB0SA1',
 'DR1MDAB0SA2',
 'DR1MDAB0SI1039',
 'DR1MDAB0SI1669',
 'DR1MDAB0SI2299',
 'DR1MDAB0SX139',
 'DR1MDAB0SX229',
 'DR1MDAB0SX319',
 'DR1MDAB0SX409',
 'DR1MDAB0SX49',
 'DR1MJSW0SA1',
 'DR1MJSW0SA2',
 'DR1MJSW0SI1010',
 'DR1MJSW0SI1640',
 'DR1MJSW0SI2270',
 'DR1MJSW0SX110',
 'DR1MJSW

In [107]:
class TimitGenerator(object):
    def __init__(self, datafile, batch_size=20, shuffle=True, mask_value=0.):
        self._datafile = datafile
        self._batch_size = batch_size
        self._keys = list(datafile.keys())
        self._batch_index = 0
        self._shuffle = True
        self._mask_value = mask_value
        
    def shuffle_keys(self):
        random.shuffle(self._keys)
        
    def next_sample(self):
        key = self._keys[self._batch_index]
        self._batch_index += 1
        if self._batch_index == len(self._keys):
            self._batch_index = 0
            if shuffle:
                self.shuffle_keys()
        return key
    
    def normalize(self, sequence):
        return (sequence - np.mean(sequence, axis=0)) / np.std(sequence, axis=0)
    
    def pad_features(self, sequence, target_length):
        return np.pad(sequence, 
                      ((0, target_length - sequence.shape[0]), (0, 0)), 'constant', 
                      constant_values=((0, self._mask_value), (0, 0)))
    
    def pad_labels(self, sequence, target_length):
        return np.pad(sequence, 
                      ((0, target_length - sequence.shape[0]), (0, 0)), 'constant', 
                      constant_values=((0, 0), (0, 0)))
            
    def generator(self):
        while True:
            batch_keys = [self.next_sample() for _ in range(self._batch_size)]
            batch_features = [self._datafile[key]['features'] for key in batch_keys]
            batch_labels = [self._datafile[key]['labels'] for key in batch_keys]
            max_length = np.max([f.shape[0] for f in batch_features])
            
            sample_weights = np.zeros((self._batch_size, max_length), dtype=int)
            for i in range(self._batch_size):
                sample_weights[i, :batch_features[i].shape[0]] = 1
            
            batch_features = np.array([self.pad_features(self.normalize(f), max_length) for f in batch_features])
            batch_labels = np.array([self.pad_labels(f, max_length) for f in batch_labels])
            
            yield batch_features, batch_labels, sample_weights
            
            

In [108]:
gen = TimitGenerator(train_data)

In [109]:
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(None, 13)))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
# TimeDistributed is nodig om het Dense deel op iedere time step toe te passen
model.add(TimeDistributed(Dense(39, activation='tanh')))
model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mse'], sample_weight_mode='temporal')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_9 (Masking)          (None, None, 13)          0         
_________________________________________________________________
lstm_21 (LSTM)               (None, None, 100)         45600     
_________________________________________________________________
dropout_11 (Dropout)         (None, None, 100)         0         
_________________________________________________________________
lstm_22 (LSTM)               (None, None, 100)         80400     
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 39)          3939      
Total params: 129,939
Trainable params: 129,939
Non-trainable params: 0
_________________________________________________________________


In [110]:
model.fit_generator(gen.generator(), steps_per_epoch=100)

Epoch 1/1
100/100 [==============================] - 157s 2s/step - loss: 0.0321 - mean_squared_error: 0.0203


In [111]:
X1 = test_data['DR1FAKS0SA1']['features']
y1 = test_data['DR1FAKS0SA1']['labels']

In [123]:
pred = model.predict(np.expand_dims(X1, axis=0))[0]
phones = np.argmax(pred, axis=1)

In [132]:
phones

array([28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 36,
       36, 36, 36, 15, 27,  6,  6,  6,  6,  6,  6, 15, 15,  1, 27,  6,  6,
        6, 16, 16, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 29, 29, 17, 11, 11, 11, 11, 11, 11, 11, 11, 17, 17, 17, 17, 17,
       17, 17, 17, 17,  4, 15, 15, 15,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16,  1,  1,  1, 17,
       17, 17, 17, 17, 17, 17, 17,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, 11, 11, 11, 11, 11, 11, 11, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
       28, 28, 28, 28, 15, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       13, 13, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 28,
       28, 28, 28, 28, 28

In [133]:
np.argmax(y1, axis=1)

array([30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 15, 15, 15, 15, 15,
       15, 15,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 30, 30,
       30, 30, 30,  7, 37, 37, 37, 11, 11, 11, 11, 11, 11, 11, 11, 30, 30,
       30, 30, 30, 30,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 27, 27,
       27, 27, 27, 27, 27, 30, 30, 30, 30, 30, 30, 30, 19, 19, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 34, 34, 34, 34, 34, 34,
       34, 34, 34, 34, 34, 34, 34, 34,  9,  9,  9,  9, 16, 16, 16, 16, 16,
       16, 16, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 30, 30, 30, 30, 14,
       14, 14, 14, 27, 27, 27, 27, 17, 17, 17, 17, 17, 17, 17, 17, 28, 28,
       28, 28, 28, 28, 28